In [ ]:
# リスト 4.2.1 NLU呼び出し用インスタンス生成（説明コメント付き・.envから資格情報読込版）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の Python SDK を用いた
#     クライアント(NaturalLanguageUnderstandingV1)の初期化を行う。
#   ・APIキー/URL はハードコードせず、.env（またはOS環境変数）から安全に取得する。
#
# セキュリティ/運用の要点:
#   ・ソースコード/リポジトリに認証情報をベタ書きしない（.env を .gitignore に追加する）。
#   ・本番/開発で異なる資格情報を使い分ける場合、.env を切り替えるだけで済む構成にする。
#   ・API バージョンは後方互換に注意。必要に応じて環境変数で上書き可能にしておく。
#
# 事前準備:
#   1) python-dotenv を使用する場合はインストール:
#        pip install python-dotenv
#   2) プロジェクト直下に .env を作成（例）
#        WATSON_NLU_APIKEY=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#        WATSON_NLU_URL=https://api.jp-tok.natural-language-understanding.watson.cloud.ibm.com/instances/xxxxxxxx
#        # （任意）バージョンを固定したい場合
#        # WATSON_NLU_VERSION=2021-08-01
#
#   3) .gitignore に以下を追加して .env をコミットしない:
#        .env
# =============================================================================

import os

# --- .env 読み込み（存在すれば） ------------------------------------------------
try:
    from dotenv import load_dotenv

    # override=False: 既にOS環境に同名キーがある場合は .env を優先しない（誤上書きを防ぐ）
    load_dotenv(override=False)
except ModuleNotFoundError:
    # python-dotenv 未導入でも、OS環境変数から直接取得できるため致命的エラーにはしない
    pass

# --- 環境変数から資格情報を取得 -------------------------------------------------
#   ・キー名の冗長化: 利用環境により命名が違っても拾えるようフォールバックを用意
NLU_APIKEY = (
    os.getenv("WATSON_NLU_APIKEY")
    or os.getenv("NLU_APIKEY")
    or os.getenv("IBM_WATSON_NLU_APIKEY")
)

NLU_URL = (
    os.getenv("WATSON_NLU_URL")
    or os.getenv("NLU_URL")
    or os.getenv("IBM_WATSON_NLU_URL")
)

# 任意: バージョンは環境変数で上書き可能。指定が無ければ従来互換のデフォルトにする
NLU_VERSION = os.getenv("WATSON_NLU_VERSION", "2019-07-12")
# ※ IBM の推奨最新版に合わせたい場合は適宜更新（例: "2021-08-01"）。SDK/サービス側の互換に注意。

# --- バリデーション（早期失敗で原因を明確化） ---------------------------------
missing = []
if not NLU_APIKEY:
    missing.append("WATSON_NLU_APIKEY")
if not NLU_URL:
    missing.append("WATSON_NLU_URL")

if missing:
    raise RuntimeError(
        "NLU の資格情報が不足しています。以下の環境変数を .env または OS 環境に設定してください: "
        + ", ".join(missing)
    )

# --- IBM Watson NLU SDK の初期化 ------------------------------------------------
# SDKのimportは資格情報の検証後に行ってもよい（起動時エラーの責務分離）
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# 旧スタイルのサブモジュール（v1のエクスポート）は必要に応じて:
# from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions, EntitiesOptions

# 認証オブジェクト（IAM）
authenticator = IAMAuthenticator(NLU_APIKEY)

# クライアント生成:
#  - version: API 呼び出しのバージョン日付。サービス仕様と整合する値を用いること。
#  - authenticator: IAMAuthenticator を渡す。
nlu = NaturalLanguageUnderstandingV1(version=NLU_VERSION, authenticator=authenticator)

# サービスURLの設定（インスタンスごとに異なる）
nlu.set_service_url(NLU_URL)

# --- 動作確認の雛形（任意・コメントアウト） -----------------------------------
# from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions
# try:
#     resp = nlu.analyze(
#         text="今日はいい天気ですね。Watsonでキーワード抽出を試します。",
#         features=Features(keywords=KeywordsOptions(limit=3))
#     ).get_result()
#     # print(resp)  # 返却JSON（実運用ではログに個人情報を出さないこと）
# except Exception as e:
#     # ネットワーク/認証/権限/リージョン誤りなどの例外をここで検知
#     raise

In [ ]:
# リスト 4.2.2 NLU呼び出し用共通関数（詳細コメント付き）
# =============================================================================
# 役割:
#   - 事前に初期化済みの IBM Watson NLU クライアント（変数: nlu）を用いて
#     テキスト解析 API を呼び出し、返却 JSON から関心のキーのみを取り出す。
#
# 使い方の要点（理論・API設計の観点）:
#   - IBM Watson NLU では「どの分析を実行するか」を Features オブジェクトで宣言する
#     （例: EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions などを束ねる）。
#   - `nlu.analyze(...)` は HTTP 経由で非同期実行され、`.get_result()` で Python dict(JSON) を返す。
#   - 本関数はその dict から `key` で指定されたトップレベルの要素（例: "entities", "keywords"）を返す。
#     * 存在しないキーを指定すると KeyError となる（後述の安全版で緩和可能）。
#   - ネットワーク障害や認証失敗、レート制限などで例外が発生し得るため、呼び出し側で try/except を推奨。
#
# 例:
#   from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions
#   feats = Features(entities=EntitiesOptions(limit=5, sentiment=False, emotion=False))
#   entities = call_nlu("今日はいい天気ですね。", feats, key="entities")
#   # entities は List[dict]（各エンティティのスパン、タイプ、スコア等を含む）
#
# 注意（実務）:
#   - 入力テキストの長さ・言語・レート制限などサービス側の上限に留意（大きな文書は要分割）。
#   - API のバージョンやリージョンにより挙動が異なる場合があるため、.env で設定を一元管理する。
# =============================================================================

from typing import Any, Dict, List, Union

# Features 型は実行時には SDK から供給されるため、型ヒントとしての参照に留める
# （循環 import を避けるためのフォワード参照も可能）
try:
    from ibm_watson.natural_language_understanding_v1 import (
        Features,
    )  # 型ヒント用（任意）
except Exception:  # ランタイムに存在しない場合でも関数本体の実行には影響しない
    Features = Any  # フォールバック（型チェックツール向けの便宜）


def call_nlu(
    text: str, features: "Features", key: str
) -> Union[Dict[str, Any], List[Any], Any]:
    """
    IBM Watson NLU を呼び出し、返却 JSON から指定キーの要素を取り出して返す薄いヘルパー。

    引数:
        text (str): 解析対象テキスト（UTF-8想定）
        features (Features): 実行する分析機能の宣言（EntitiesOptions/KeywordsOptions 等を束ねたもの）
        key (str): 返却 JSON のトップレベルキー（例: 'entities', 'keywords', 'sentiment', 'categories' など）

    戻り値:
        任意（Union[dict, list, Any]):
            返却 JSON の `key` に対応する値（存在しないキーを指定すると KeyError）

    例外:
        - ネットワーク障害や認証エラー時に SDK 由来の例外が送出される。
        - `key` が返却 JSON に無い場合は KeyError。

    設計メモ:
        - 本関数は「最小限の責務」に留めている（バリデーション/リトライ/ログは呼び出し側で制御）。
        - 実運用では、レート制限や一時的なネットワーク失敗に備えたリトライ・指数バックオフ、
          ならびに返却スキーマ検証（pydantic/dataclasses 等）を併用することを推奨。
    """
    response = nlu.analyze(text=text, features=features).get_result()
    return response[key]


# -----------------------------------------------------------------------------
# （任意）安全版: KeyError の回避・既定キー・軽いバリデーション・簡易リトライを追加する例
# -----------------------------------------------------------------------------
# from ibm_cloud_sdk_core.api_exception import ApiException
# import time
#
# def call_nlu_safe(
#     text: str,
#     features: "Features",
#     key: str | None = None,
#     retries: int = 2,
#     backoff_sec: float = 0.8,
# ) -> Any:
#     """
#     - KeyError を避け、key 未指定時は生の JSON を返す。
#     - 一時的エラーに対して簡易リトライを行う。
#     """
#     last_err = None
#     for attempt in range(retries + 1):
#         try:
#             resp = nlu.analyze(text=text, features=features).get_result()
#             if key is None:
#                 return resp
#             return resp.get(key, None)  # 無ければ None（呼び出し側で判定しやすい）
#         except Exception as e:  # ApiException を個別に握るとより良い
#             last_err = e
#             if attempt < retries:
#                 time.sleep(backoff_sec * (2 ** attempt))  # 指数バックオフ
#                 continue
#             raise last_err
# -----------------------------------------------------------------------------

In [ ]:
# リスト 4.2.3 エンティティ抽出機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の「エンティティ抽出」を用いて、
#     テキスト中から固有表現（人物・組織・地名・日付など）を検出し、JSON 形式で確認する。
#
# 背景/理論:
#   ・エンティティ抽出 (Named Entity Recognition; NER) は、文中の名詞句をカテゴリ（PERSON, LOCATION 等）
#     にラベル付けするタスク。以降の関係抽出・イベント抽出・検索インデキシングの基礎特徴となる。
#   ・Watson NLU は言語自動判定を行うが、テキストが短い/混在言語のときは誤判定に注意。
#     必要に応じて features に `language="ja"` を付けるか、分析APIの別引数で明示指定する設計もあり。
#
# 事前準備:
#   ・既に .env から資格情報を読み込み、`nlu` クライアントが初期化済み（リスト 4.2.1）。
#   ・共通関数 `call_nlu(text, features, key)` が定義済み（リスト 4.2.2）。
#
# 出力:
#   ・`entities` キー配下に、抽出されたエンティティの配列（各要素は text, type, relevance など）を返す。
#   ・`ensure_ascii=False` で日本語を可読な形で表示。
# =============================================================================

# （補助）本セルだけで実行する場合に備えた import
#  * 既にインポート済みなら重複定義は無害
import json
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions

# 対象テキスト
# - 人物: 「安倍首相」「トランプ氏」
# - 日付/時間: 「昨日」
# - 施設/場所: 「大阪の国際会議場」
# こうした固有表現が NER の検出対象になる。
text = "安倍首相はトランプ氏と昨日、大阪の国際会議場で会談した。"

# 機能として「エンティティ抽出機能」を利用
# -----------------------------------------------------------------------------
# EntitiesOptions の主なオプション例（必要に応じて拡張）:
#   - limit: 返却件数上限
#   - mentions: エンティティの各出現箇所（スパン）も含めるか
#   - sentiment/emotion: エンティティごとの感情/極性分析を併せて実施
#   - model: カスタム NLU モデルを指定（学習済みがある場合）
# ここでは既定（全件・最小限情報）で実行する。
features = Features(
    entities=EntitiesOptions(
        # limit=10,
        # mentions=True,
        # sentiment=False,
        # emotion=False,
        # model="<your-custom-model-id>"
    )
)

# 共通関数呼び出し
# - 第3引数 "entities" は NLU 返却 JSON のトップレベルキー。
# - 例外（ネットワーク/認証/429 等）は呼び出し側で捕捉する設計（詳細はレビュー参照）。
ret = call_nlu(text, features, "entities")

# 結果の表示
# - 抽出結果は配列（List[dict]）。各要素には "type", "text", "relevance", "count",
#   場合によって "disambiguation"（同名異義の分解）などが含まれる。
# - 日本語を読みやすくするため ensure_ascii=False。
print(json.dumps(ret, indent=2, ensure_ascii=False))

# =============================================================================
# 注意/拡張ヒント:
#   ・短文で誤検出が起きる場合は、前後文脈を足す/ドメイン固有語を事前正規化する/句点での分割を調整する。
#   ・固有表現のカテゴリ粒度（例: "人物肩書" vs "人名"）はサービス側モデル依存。
#   ・後続処理（例: ES へのインデックス、グラフDB でのノード登録）では、
#     "normalized_text"（正規化表記）や "disambiguation.dbpedia_resource" 等の正規IDをキーに採用すると安定。
# =============================================================================

In [ ]:
# リスト 4.2.4 関係抽出機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の「関係抽出（Relation Extraction）」機能を用いて、
#     文中のエンティティ間の関係（主語-述語-目的語 等の関係ラベル/役割）を検出し、JSON で確認する。
#
# 背景/理論:
#   ・関係抽出は、まず NER（固有表現抽出）で候補エンティティを同定 → そのペア（または n-項）の
#     間に成り立つ意味関係をラベリングするタスクである。
#   ・一般的には (subject, relation, object) の三つ組や、argument 役割（ARG0/ARG1 等）として返る。
#   ・Watson NLU では `relations` を有効化すると、内部でエンティティ・句構造を解析し、
#     「開催地」「所属」「位置」などの関係を推定する（具体的ラベルはモデル依存）。
#
# 実務上の要点:
#   ・短文や文脈の乏しい文では関係の自信度が低くなりやすい。必要なら文脈を付与して精度を上げる。
#   ・日本語判定が揺れると解析器の選択が変わり結果が不安定になるため、必要に応じて language="ja" を
#     明示指定する（本スニペットは共通関数 call_nlu をそのまま使うため引数追加はしていない）。
#   ・返却 JSON はモデル/バージョンで微細に変化し得るため、下流でのキー存在チェックを推奨。
#
# オプション設計のヒント:
#   ・RelationsOptions には（環境により）`model` などの指定が可能（独自学習モデルがある場合に利用）。
#   ・関係抽出は NER の質に依存するため、同時に EntitiesOptions(mentions=True) で可視化すると
#     デバッグしやすい（どのスパンが関係の対象になったか追跡できる）。
#
# 依存:
#   ・前段で NLU クライアント `nlu` が初期化済み（.env から資格情報読込）であること（リスト 4.2.1）。
#   ・共通関数 `call_nlu(text, features, key)` が定義済みであること（リスト 4.2.2）。
# =============================================================================

# 最小限の import（このセル単体での実行にも耐えるようにしておく）
import json
from ibm_watson.natural_language_understanding_v1 import Features, RelationsOptions

# ※ call_nlu は前段で定義済みの想定。未定義なら call_nlu_safe を用意するか、nlu.analyze を直接呼ぶ。

# 対象テキスト
# - エンティティ候補: 「このイベント」（抽象イベント）、「東京」（地名）、「国立競技場」（施設）
# - 期待される関係例: "開催地(location_of_event)" や "場所(at)" に相当する関係ラベル（モデル依存）
text = "このイベントは東京の国立競技場で開催されました。"

# 機能として「関係抽出機能」を利用
# -----------------------------------------------------------------------------
# RelationsOptions の主な指定例（必要に応じて利用）:
#   - model="<custom-model-id>"   : カスタム学習済み関係抽出モデルがある場合に指定
#   - language="ja"               : analyze 引数に渡すのが一般的（call_nlu を拡張するか、直接呼び出す）
# ここでは既定の関係抽出を用いる。
features = Features(
    relations=RelationsOptions(
        # model="<your-custom-relations-model-id>"  # 例: カスタムモデルがある場合
    )
)

# 共通関数呼び出し
# - "relations" キー配下に、検出された関係の配列が返る。
# - 各要素は "type"（関係ラベル）、"arguments"（関係を構成するエンティティ/スパンと役割）、
#   "sentence"（対象文）などを含むことが多い（モデル/バージョンに依存）。
ret = call_nlu(text, features, "relations")

# 結果の表示
# - ensure_ascii=False: 日本語を可読表示。
# - 返却例イメージ（参考/実行環境で変動）:
#   [
#     {
#       "type": "located_at",
#       "sentence": "このイベントは東京の国立競技場で開催されました。",
#       "arguments": [
#         {"text": "このイベント", "entities": [...], "location": {...}, "role": "subject"},
#         {"text": "国立競技場",   "entities": [...], "location": {...}, "role": "object"}
#       ]
#     }
#   ]
print(json.dumps(ret, indent=2, ensure_ascii=False))

# =============================================================================
# 注意/拡張:
#   ・誤検出/過検出が見られる場合:
#       - 文を短く分割しすぎない（係り受け情報が失われる）
#       - 補足文脈を足す（主語省略の補完）
#       - NER の mentions を併用してスパンを確認（関係の引数抽出が適切か検証）
#   ・スキーマの安定運用:
#       - 下流では "type" と "arguments[*].role/text" を必須キーとして扱い、欠落時はスキップ/要再解析。
#       - ログには原文を残さず、必要最小限のメタ情報（関係タイプ、引数の正規表記）を記録。
#   ・評価:
#       - 開発コーパスで関係タイプごとの適合率/再現率/F1 を算出し、文体差（ニュース/ブログ/広報）に対する頑健性を確認。
# =============================================================================

In [ ]:
# リスト 4.2.5 評判分析機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson NLU の「評判（感情極性）分析」を用いて、日本語テキストの
#     文書レベル極性（positive/negative/neutral など）とスコアを取得・確認する。
#
# 理論メモ（なぜこうなるか）:
#   ・評判分析（sentiment analysis）は、文の語彙・係り受け・否定表現などを手掛かりに
#     「極性（valence）」を推定するタスク。Watson NLU では学習済みのモデルにより
#     文書/文単位での極性ラベルとスコア（連続値）が返る。
#   ・日本語短文では言語自動判定が揺れやすいため、可能なら analyze 時に language="ja"
#     を明示するのが理論的に安定（本スニペットは共通関数 call_nlu をそのまま用いる）。
#   ・対象（target）を指定すれば、同一文書内の特定対象に対する極性（targeted sentiment）
#     を得られる（例: “ソニー”, “音”, “写真” など）。ここでは文書全体の極性のみ。
#
# 事前条件:
#   ・4.2.1 で .env から資格情報を読み込み、 nlu クライアントを初期化済み。
#   ・4.2.2 の共通関数 call_nlu(text, features, key) が定義済み。
# =============================================================================

import json
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions

# テキスト1 (ポジティブな評判の例)
# - 「さすが」「写りもいい」「音がまた良い」など肯定表現が多く、正極性が期待される。
text1 = "さすがはソニーです。写真の写りもいいですし、音がまた良いです。"

# テキスト2 (ネガティブな評判の例)
# - 「残念ながら」「合わず」「利用できませんでした」など否定・失望の語彙が多く、負極性が期待。
text2 = "利用したかったアプリケーションは、残念ながらバージョン、性能が合わず、利用できませんでした。"

# -----------------------------------------------------------------------------
# 評判分析（文書レベル）
# - SentimentOptions(): 既定では文書レベルの極性を返す。
# - targets を使うと対象語ごとの極性も返せる（例: SentimentOptions(targets=["ソニー","音"]))
# - 呼び出し側で language="ja" を渡せる設計なら、日本語モデル固定がより安定。
# -----------------------------------------------------------------------------

# テキスト1を評判分析
features = Features(sentiment=SentimentOptions())
ret = call_nlu(text1, features, "sentiment")
print(json.dumps(ret, indent=2, ensure_ascii=False))
# 返却例イメージ:
# {
#   "document": { "label": "positive", "score": 0.9 },
#   "targets": [...]  # targets を指定した場合のみ
# }

# テキスト2を評判分析
features = Features(sentiment=SentimentOptions())
ret = call_nlu(text2, features, "sentiment")
print(json.dumps(ret, indent=2, ensure_ascii=False))
# 返却例イメージ:
# {
#   "document": { "label": "negative", "score": -0.7 }
# }

# =============================================================================
# 注意/拡張:
#   ・日本語では否定のスコープ（「〜は良いが、…は悪い」）や婉曲表現が極性推定を難しくする。
#   ・対象別の評価が必要なら targets を利用（例: SentimentOptions(targets=["音","写真"]))。
#   ・大量処理時はレート制限（429）や一時障害（5xx）に備え、呼び出し側で指数バックオフを実装。
#   ・ログに原文を残さない（PII/著作権配慮）。必要なら要約やハッシュ化したメタ情報のみ記録。
# =============================================================================

In [ ]:
# リスト 4.2.6 キーワード抽出機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の「キーワード抽出」を用いて、
#     テキスト内の重要語（keywords）を抽出し、重要度（relevance）等を確認する。
#
# 理論メモ:
#   ・Watson NLU の keywords は「表層句の重要度推定」に近いタスクで、固有表現抽出（entities）や
#     概念抽出（concepts）とは役割が異なる。文章の要旨把握やタグ付け、検索クエリ拡張に有効。
#   ・日本語では分かち書き/句抽出の精度が結果を左右するため、可能なら analyze の language を
#     "ja" に固定して日本語モデルを確実に使うと安定する（下の call_nlu は最小版のため直接は指定せず）。
#
# 事前条件:
#   ・.env から資格情報を読み込み、`nlu` クライアントが初期化済み（リスト 4.2.1）。
#   ・共通関数 `call_nlu(text, features, key)` が定義済み（リスト 4.2.2）。
# =============================================================================

# 最低限の import（このセル単体実行にも耐えるよう冪等に記述）
import json
from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions

# 対象テキスト
# - 行継続（\）で 1 つの文字列に連結している。三連クォートの複数行文字列でも可。
# - 温泉説明文は属性（泉質・効能・運用方法）が多く、keywords と相性が良い題材。
text = "ながぬま温泉は北海道でも屈指の湯量を誇り,\
加水・加温はせずに100%源泉掛け流しで、\
保温効果が高く湯冷めしにくい塩化物泉であり、\
「熱の湯」とも呼ばれ、保養や療養を目的として多くの方が訪れている。"

# 機能として「キーワード抽出機能」を利用
# -----------------------------------------------------------------------------
# KeywordsOptions の主なオプション:
#   - limit: 返却件数上限
#   - sentiment, emotion: キーワード単位で感情/極性推定も付与（説明責任が必要な場合に有用）
#   - stemming/lemmatization は内部モデル依存（日本語は分割精度の影響が大きい）
# ここでは上位 5 件を抽出する最小構成。
features = Features(
    keywords=KeywordsOptions(
        limit=5,
        # sentiment=False,  # 必要に応じてキーワードごとの極性を付与
        # emotion=False     # 必要に応じてキーワードごとの感情分布を付与
    )
)

# 共通関数呼び出し
# - 第3引数 "keywords" は返却 JSON のトップレベルキー名。
# - 返却は List[dict] が想定され、各要素に "text", "relevance", "count" などが含まれる。
#   * "relevance": 重要度スコア（0〜1 付近の連続値、絶対指標ではなく相対比較に用いる）
#   * "count": 文書内出現回数（分かち書き・正規化のされ方で変動）
ret = call_nlu(text, features, "keywords")

# 結果の表示
# - ensure_ascii=False により日本語を可読表示。
# - 実運用ではログに原文全文を残さず、抽出語とスコアのみを構造化記録することを推奨（PII/著作権配慮）。
print(json.dumps(ret, indent=2, ensure_ascii=False))

# =============================================================================
# 注意/拡張:
#   ・language 固定: `call_nlu` を拡張して `language="ja"` を透過的に渡せるようにすると安定。
#   ・前処理: 記号（「」・全角記号・％ 等）の正規化でキーワード境界が改善することが多い。
#   ・評価: タグ付け/検索用途では、再現率よりも適合率（トピック要約の妥当性）を重視して limit/閾値を調整。
#   ・targets との併用: 目的語や特定属性（例: 「泉質」「効能」）を別途パターン抽出し、keywords と結合すると説明性が上がる。
# =============================================================================

In [ ]:
# リスト 4.2.7 概念分析機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の「概念抽出（Concepts）」を用いて、
#     テキストに内在する一般概念（知識グラフ上のノードに対応しやすい）を取得する。
#
# 理論メモ（keywords/entities との違い）:
#   ・keywords: 文書内で相対的重要度が高い語句（表層句のランキング）。
#   ・entities: 命名対象（人物・組織・地名など）の固有表現（できればKBに紐づく）。
#   ・concepts: 文書の意味内容を上位概念へマッピングした結果（DBpedia/Wikipedia等の知識ベースに
#               対応しやすい抽象概念）。タグ付け・類似度計算・ナレッジリンクに適する。
#
# 再現性・安定性のポイント:
#   ・日本語短文/固有ドメインでは自動言語判定が揺れやすい。可能なら analyze 時に language="ja" を明示。
#   ・本スニペットは共通関数 call_nlu を利用（リスト 4.2.2）。その実装に language を渡す拡張余地あり。
#   ・全角/半角・余分な空白は概念マッチの安定性に影響しうる（正規化は下の「任意の前処理」を参照）。
#
# 依存関係:
#   ・.env から資格情報を読み込み、NLU クライアント nlu を初期化済み（リスト 4.2.1）。
#   ・call_nlu(text, features, key) が定義済み（リスト 4.2.2）。
# =============================================================================

# 冪等な import（このセル単体でも動作するよう補助的に記述）
import json
from ibm_watson.natural_language_understanding_v1 import Features, ConceptsOptions

# 対象テキスト
# - 行継続（\）で 1 つの文字列に連結。
# - 本文中に「源泉 掛け流し」「呼ば れ」などスペースが混入しているため、
#   概念マッチの安定性を上げたい場合は事前に空白正規化を検討（下の任意前処理を参照）。
text = "ながぬま温泉は北海道でも屈指の湯量を誇り,\
加水・加温はせずに100%源泉 掛け流しで、\
保温効果が高く湯冷めしにくい塩化物泉であり、\
「熱の湯」とも呼ば れ、保養や療養を目的として多くの方が訪れている。"

# （任意）前処理の例: Unicode 正規化＋空白統一
# -------------------------------------------------
# import re, unicodedata
# text_norm = unicodedata.normalize("NFKC", text)   # 全角/半角のゆらぎ補正
# text_norm = re.sub(r"\s+", " ", text_norm)        # 空白を単一スペースへ
# ※ 使用する場合は、call_nlu の第1引数を text_norm に置き換える。

# 機能として「概念分析機能」を利用
# - ConceptsOptions(limit=3): 上位3件の概念を返す（relevance 付き）。
#   返却要素には "text", "relevance", "dbpedia_resource" 等（環境により異なる）が含まれることが多い。
features = Features(concepts=ConceptsOptions(limit=3))

# 共通関数呼び出し
# - 第3引数 "concepts" は NLU 返却 JSON のトップレベルキー名。
# - 内部で nlu.analyze(..., features=features) が呼ばれ、概念の配列を返す想定。
ret = call_nlu(text, features, "concepts")

# 結果の表示
# - ensure_ascii=False により日本語を可読表示。
# - 実運用ログでは原文を保存せず、概念の URI/ラベル/スコア/バージョンのみを構造化保存することを推奨。
print(json.dumps(ret, indent=2, ensure_ascii=False))

# =============================================================================
# 注意/拡張:
#   ・language 固定: call_nlu を拡張し language="ja" を渡すとモデル選択が安定。
#   ・KB 連携: 返却の dbpedia_resource/Wikipedia URL を知識グラフの ID として利用し、
#              検索・推薦・類似度計算（概念ベース）に活用できる。
#   ・評価: 人手アノテーションの「想定概念」集合を用意し、Exact/Partial Match（リンク一致/語一致）で
#            適合率・再現率・F1 を測定。表記ゆれには正規化後評価も併記すると良い。
#   ・レート/障害: 429/5xx への指数バックオフ、KeyError 等スキーマ差分のハンドリングを上位で実装。
# =============================================================================

In [ ]:
# リスト 4.2.8 カテゴリ分類機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の「カテゴリ分類（Categories）」を使用し、
#     入力テキストがどのトピック階層（カテゴリ）に属するかを推定する。
#
# 理論メモ:
#   ・Categories は文書全体の「話題（トピック）」を階層ラベル（例: "/travel/tourism" のようなパス）として返す。
#   ・返却要素には通常 "label"（階層カテゴリ）と "score"（信頼度に類する連続値）が含まれる。
#   ・keywords（重要語抽出）や entities（固有表現）と異なり、「全体主題」の抽象化が目的。
#   ・日本語短文では文脈不足によりカテゴリの確度が低下しやすいため、可能なら analyze 時に language="ja" を
#     明示して日本語モデルに固定し、文量/文脈を増やすと安定しやすい（本スニペットでは関数仕様上そのまま呼ぶ）。
#
# 事前条件:
#   ・4.2.1 にて .env から API キー/URL を読み込み、`nlu` クライアントを初期化済み。
#   ・4.2.2 の共通関数 `call_nlu(text, features, key)` が定義済み。
#   ・本スニペットは「コード追加変更なし」で説明コメントのみを付与している。
# =============================================================================

# 最低限の import（このセル単体での実行にも耐えるよう冪等に記述）
import json
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions

# 対象テキスト
# - 行継続（\）で 1 つの文字列に連結している。三連クォートによる複数行文字列でも可。
# - 内容は「温泉施設の整備・保養」に関する記述で、観光/地域/公共政策などのカテゴリが期待される。
text = "自然環境の保護を図るとともに、地域に調和した温泉利用施設を維持整備し、\
豊かさとふれあいのある保養の場とする。"

# 機能として「カテゴリ分類機能」を利用
# -----------------------------------------------------------------------------
# CategoriesOptions:
#   - limit（環境によって利用可）で上位件数を制御できる場合があるが、ここでは既定値のまま。
#   - 返却は一般に上位数件の {label, score} を階層順に含む。
features = Features(categories=CategoriesOptions())

# 共通関数呼び出し
# - 第3引数 "categories" は NLU 返却 JSON のトップレベルキー名。
# - 内部で nlu.analyze(text=text, features=features) が呼ばれ、カテゴリ配列が返る想定。
# - 再現性を重視する場合は、call_nlu を拡張して language="ja" を透過的に渡せるようにすると安定。
ret = call_nlu(text, features, "categories")

# 結果の表示
# - ensure_ascii=False: 日本語を可読表示。
# - 返却例のイメージ（環境依存）:
#   [
#     {"label": "/travel/tourism", "score": 0.85},
#     {"label": "/health/wellness", "score": 0.63},
#     ...
#   ]
print(json.dumps(ret, indent=2, ensure_ascii=False))

# =============================================================================
# 注意/拡張:
#   ・短文対策: 文量を増やす/複数文を与えると主題の抽出が安定。
#   ・前処理: Unicode 正規化・余計な空白/記号の統一は分割と主題推定の一貫性に寄与。
#   ・スコア解釈: score は相対的連続値。k 件選出/しきい値二値化の基準は手元データで較正する。
#   ・評価: 人手付与のトピックラベルと比較し、トップ1/トップk 精度、階層的一致（上位ノード一致）を指標化。
#   ・ロギング: 原文の保存は避け、text のハッシュ・返却カテゴリ・スコア・language・バージョンを構造化保存。
# =============================================================================

In [ ]:
# リスト 4.2.9 意味役割分析（Semantic Roles）機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の「意味役割分析」を用いて、
#     文内の述語（predicate）とその項（arguments: 主語/目的語/時間など）を抽出する。
#
# 背景/理論:
#   ・意味役割分析（SRL: Semantic Role Labeling）は、(述語, 役割付き引数) の構造化表現を返すタスク。
#     例: 「IBM は 毎年 多くの特許 を 取得する」
#          → 述語: 取得する（action）
#             主語/施事: IBM（subject/agent）
#             目的語/被影響体: 特許（object/patient）
#             時間状況: 毎年（temporal）
#   ・SRL は NER（固有表現抽出）や依存構造解析の結果に依存しやすく、言語モデルの選択（日本語固定）
#     と文の自然性（助詞・語順・句読点の適切さ）が精度に影響する。
#
# 実装上のポイント:
#   ・短文や外来語混在では自動言語判定が揺れやすい。可能なら analyze で language="ja" を明示。
#   ・SemanticRolesOptions には、返却の詳細度を調整するオプションがある（環境により利用可）:
#       - entities=True/False     : 役割スパンに紐づくエンティティ情報を併せて返す
#       - keywords=True/False     : キーワードも併せて返す
#       - limit=…                 : 返却する述語の最大数
#   ・返却 JSON はモデル/バージョンで微差があるため、下流でキー存在チェックを推奨。
#
# 依存:
#   ・前段で NLU クライアント `nlu` の初期化（.env からの資格情報読込を含む）が完了している（リスト 4.2.1）。
#   ・共通関数 `call_nlu(text, features, key)` が定義済み（リスト 4.2.2）。
# =============================================================================

# （このセル単体でも実行できるよう、最小限の import を明示）
import json
from ibm_watson.natural_language_understanding_v1 import Features, SemanticRolesOptions

# 対象テキスト
# - SRL 的には以下の役割が期待される（モデル依存）:
#     subject/agent: 「IBM」
#     action/predicate: 「取得しています」（基本形「取得する」）
#     object/patient: 「特許」
#     temporal: 「毎年」
text = "IBMは毎年、多くの特許を取得しています。"

# 機能として「意味役割分析機能」を利用
# -----------------------------------------------------------------------------
# 必要に応じて詳細化の例（コメントアウト）:
# features = Features(semantic_roles=SemanticRolesOptions(entities=True, keywords=False, limit=3))
features = Features(semantic_roles=SemanticRolesOptions())

# 共通関数呼び出し
# - 第3引数 "semantic_roles" は返却 JSON のトップレベルキー名。
# - call_nlu を拡張済みなら language="ja" を渡して日本語モデルを固定すると安定:
#     ret = call_nlu(text, features, "semantic_roles", language="ja")
ret = call_nlu(text, features, "semantic_roles")

# 結果の表示
# - ensure_ascii=False で日本語可読。返却例（環境依存・参考）:
#   [
#     {
#       "sentence": "IBMは毎年、多くの特許を取得しています。",
#       "subject": [{"text": "IBM"}],
#       "action":  {"text": "取得しています", "normalized": "取得する"},
#       "object":  [{"text": "多くの特許"}],
#       "temporal": [{"text": "毎年"}]
#     }
#   ]
print(json.dumps(ret, indent=2, ensure_ascii=False))

# =============================================================================
# 注意/拡張:
#   ・短文化対策: 文脈を1–2文追加すると役割推定の信頼度が上がりやすい。
#   ・前処理: Unicode 正規化（NFKC）や空白統一でトークナイズの一貫性を高める。
#   ・スキーマ安定化: 下流では (sentence, action.normalized, subject[].text, object[].text, temporal[].text)
#     を抽出するユーティリティを用意し、欠落時はスキップ/再解析ポリシーを定義。
#   ・評価: 小規模ゴールドで 役割別 F1（subject/object/action/temporal）と文タイプ別（受身/能動）の
#     ロバスト性を計測。否定表現や連体修飾の影響もスライス評価するとよい。
# =============================================================================